In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import string
import h5py
import matplotlib
from collections import Counter
# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline


pixel_depth = 255.0  # Number of levels per pixel.
screen_width = 800
screen_height = 600

In [ ]:
def get_attr(c, i, attr):
    d = c[c['digitStruct']['bbox'][i][0]][attr].value.squeeze()
    if d.dtype == 'float64':
        return d.reshape(-1)
    return np.array([c[x].value for x in d]).squeeze()

def get_label(c, i):
    d = c[c['digitStruct']['name'][i][0]].value.tostring()
    return d.replace('\x00', '')

def load_data(path):
    c = h5py.File(path)
    images = a = np.ndarray(shape=(c['digitStruct']['name'].shape[0], ), dtype='|S15')
    labels = np.zeros((len(c['digitStruct']['bbox']), 6), dtype=float)
    labels.fill(10)
    tops = np.zeros((len(c['digitStruct']['bbox']), 6), dtype=float)
    heights = np.zeros((len(c['digitStruct']['bbox']), 6), dtype=float)
    widths = np.zeros((len(c['digitStruct']['bbox']), 6), dtype=float)
    lefts = np.zeros((len(c['digitStruct']['bbox']), 6), dtype=float)
    for i in xrange(c['digitStruct']['name'].shape[0]):
        images[i] = get_label(c, i)
        l = get_attr(c, i, 'label')
        t = get_attr(c, i, 'top')
        h = get_attr(c, i, 'height')
        w = get_attr(c, i, 'width')
        le = get_attr(c, i, 'left')
        
        labels[i, :l.shape[0]] = l
        tops[i, :t.shape[0]] = t
        heights[i, :h.shape[0]] = h
        widths[i, :w.shape[0]] = w
        lefts[i, :le.shape[0]] = le
    
        if (i % 10000 == 0):
            print(i, "elapsed")
    
    return labels, images, tops, heights, widths, lefts

In [7]:
#train__tuple = load_data('train/digitStruct.mat')
#test__tuple = load_data('test/digitStruct.mat')
extra__tuple = load_data('extra/digitStruct.mat')

KeyError: "Unable to open object (Can't insert duplicate key)"

In [4]:
def maybe_pickle(struct, force=False):
    if os.path.exists(struct + '.pickle') and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % struct)
    else:
      print('Pickling %s.' % struct + '.pickle')
      permutation = np.random.permutation(extra__tuple[1].shape[0])[:2000]
      dataset = {
            'train': {
                'labels': train__tuple[0],
                'images': train__tuple[1],  
                'tops': train__tuple[2],
                'heights': train__tuple[3],
                'widths': train__tuple[4],
                'lefts': train__tuple[5],
                
                
            }, 
            'test': {
                'labels': test__tuple[0],
                'images': test__tuple[1],  
                'tops': test__tuple[2],
                'heights': test__tuple[3],
                'widths': test__tuple[4],
                'lefts': test__tuple[5],
            },
            'extra': {
                'labels': extra__tuple[0],
                'images': extra__tuple[1],  
                'tops': extra__tuple[2],
                'heights': extra__tuple[3],
                'widths': extra__tuple[4],
                'lefts': extra__tuple[5],
            },
            'valid': {
                'labels': extra__tuple[0][permutation],
                'images': extra__tuple[1][permutation],  
                'tops': extra__tuple[2][permutation],
                'heights': extra__tuple[3][permutation],
                'widths': extra__tuple[4][permutation],
                'lefts': extra__tuple[5][permutation],
            }
      }
      try:
        with open( struct + '.pickle', 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to',  struct + '.pickle', ':', e)
  
    return  struct + '.pickle'

In [5]:
extra__tuple[0][[1,2,3,4]].shape

NameError: name 'extra__tuple' is not defined

In [ ]:
maybe_pickle('svhn')

In [ ]:
with open('svhn.pickle', 'rb') as f:
        dataset = pickle.load(f)

In [ ]:
c = Counter(np.sum(dataset["train"]["tops"] > 0, axis =1))
l = plt.bar(c.keys(), c.values(), 1, color="blue")

plt.xlabel('Number of characters')
plt.ylabel('Sample Size')
plt.grid(True)
plt.title('Training Dataset')

plt.show()

c = Counter(np.sum(dataset["test"]["tops"] > 0, axis =1))
l = plt.bar(c.keys(), c.values(), 1, color="blue")

plt.xlabel('Number of characters')
plt.ylabel('Sample Size')
plt.grid(True)
plt.title('Test Dataset')

plt.show()

c = Counter(np.sum(dataset["extra"]["tops"] > 0, axis =1))
l = plt.bar(c.keys(), c.values(), 1, color="blue")

plt.xlabel('Number of characters')
plt.ylabel('Sample Size')
plt.grid(True)
plt.title('Test Dataset')

plt.show()

In [ ]:
np.std(np.max(dataset["test"]["heights"], axis=1)), np.mean(np.max(dataset["test"]["heights"], axis=1)), \
np.median(np.max(dataset["test"]["heights"], axis=1))

In [ ]:
hist = np.histogram(dataset["train"]["heights"] +dataset["train"]["tops"], bins=20)
y = hist[1][1:]
l = plt.plot(hist[0][:], y, linewidth=1)

plt.xlabel('Number of samples')
plt.ylabel('Heights + Tops')
plt.grid(True)

plt.show()

hist = np.histogram(dataset["train"]["widths"] +dataset["train"]["lefts"], bins=20)
y = hist[1][1:]
l = plt.plot(hist[0][:], y, linewidth=1)

plt.xlabel('Number of samples')
plt.ylabel('Widths + Lefts')
plt.grid(True)

plt.show()

hist = np.histogram(dataset["extra"]["widths"] +dataset["extra"]["lefts"], bins=20)
y = hist[1][1:]
l = plt.plot(hist[0][:], y, linewidth=1)

plt.xlabel('Number of samples')
plt.ylabel('Widths + Lefts')
plt.grid(True)

plt.show()